Code for scraping from indeed.de

In [1]:
#import packages

import requests, bs4, time
import pandas as pd
import os.path
from datetime import datetime

In [2]:
path = os.getcwd()
parent_folder, current_folder = os.path.split(path)

In [3]:
#define functions for parsing HTML

def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span",attrs={"class":"salaryText"}).text)
        except:
            salaries.append("Nothing_found")
    return(salaries)

def extract_location_from_result(soup): 
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find("span", attrs={"class": "location accessible-contrast-color-location"}).text)
        except:
            locations.append("Nothing_found")
   
    return(locations)

def extract_description_from_result(soup): 
    description = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            description.append(div.find("div", attrs={"class": "summary"}).text)
        except:
            description.append("Nothing_found")
   
    return(description)

def extract_date_from_result(soup): 
    date = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            date.append(div.find("span", attrs={"class": "date"}).text)
        except:
            date.append("Nothing_found")
   
    return(date)

def extract_company_from_result(soup): 
    company = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            company.append(div.find("span", attrs={"class": "company"}).text)
        except:
            company.append("Nothing_found")
   
    return(company)

def extract_links(soup):
    links =[]
    for div in soup.find_all(name='a', attrs={'class':'jobtitle turnstileLink'}):
        links.append('https://de.indeed.com'+str(div['href']))
    return links

def extract_full_desc(soup):
    text=[x.text for x in soup.find_all(name="div",attrs={"id":"jobDescriptionText"})]
    return text


def extract_headlines_from_result(soup): 
    headlines = pd.DataFrame(columns= ["location","type","salary"])
    list=[x.text for x in soup.find_all(name="span",attrs={"class":"jobsearch-JobMetadataHeader-iconLabel"})]
    try:
        headlines['location']
    except:
        salaries.append("Nothing_found")
    return(salaries)


In [12]:
#scraping code:

#decide what search term to use for finding jobs
searchTerm="data"

#create empty data frame with column headers
ads=pd.DataFrame(columns=['company','title','salary','location','description','date','full_description'])

# loop for scraping

for i in range(461, 550): # range(0:1000)
    company = []
    job_title = []
    description = []
    salary = []
    location = []
    date = []
    full_description = []
    text_list = []
    
    time.sleep(1) #ensuring at least 1 second between page grabs
    url = "https://de.indeed.com/Jobs?q="+searchTerm+"&filter=0&start="+str(i)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.content)
    df = pd.DataFrame(columns=['company','title','salary','location','description','date','full_description'])
    df['company'] = extract_company_from_result(soup)
    df['title'] = extract_job_title_from_result(soup)
    df['salary'] = extract_salary_from_result(soup)
    df['location'] = extract_location_from_result(soup)
    df['description'] = extract_description_from_result(soup)
    df['date'] = extract_date_from_result(soup)
    
    sub_urls=extract_links(soup)
    text=[]
    deets=[]
    for j in sub_urls:
        res_sub = requests.get(j)
        soup_sub = bs4.BeautifulSoup(res_sub.content)
        desc=extract_full_desc(soup_sub)
        text_list.append(desc)
        
    df['full_description'] = text_list

    ads = ads.append(df, ignore_index=True)

today = datetime.now().strftime('%Y_%m_%d_%H_%M')
ads.to_csv(parent_folder+'/data/indeed_de_'+today+'.csv', index=True, sep='\t')

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [13]:
ads

,company,title,salary,location,description,date,full_description
0,\n\nUniversitätsklinikum Carl Gustav Carus Dre...,Data Warehouse Entwickler (w/m/d),Nothing_found,Nothing_found,\nEntwicklung von ETL-Prozessen zur Integratio...,Nothing_found,[Jobs mit Aussicht\nWir sorgen für eine zuverl...
1,\nUNICEPTA GmbH,Data Analyst (m/w/d) Analytics & Insights in K...,Nothing_found,Nothing_found,\nDu arbeitest eng mit den Consultants und Dat...,Nothing_found,[Deine Aufgaben:\nMit Deiner Arbeit bist Du ma...
2,\nAnalytics Academy - The Information Lab Deut...,"Analytics Academy - Junior Data Analyst, m/w/d",Nothing_found,Nothing_found,\nDie Analytics Academy bildet die nächste Gen...,Nothing_found,[Die Analytics Academy– dein Karriere-Sprungbr...
3,\n\nMDM Münzhandelsgesellschaft mbH & Co. KG D...,Junior Data Warehouse Entwickler (m/w/d),Nothing_found,Nothing_found,"\nOptimierung, Weiterentwicklung und Wartung d...",Nothing_found,[Du hast eine Affinität zu Daten und Analysen ...
4,\nManagement Services Helwig Schmitt GmbH,Technical Data Analyst (m/w/d) - in Hofgeismar,Nothing_found,Nothing_found,\nIn einem der größten und anspruchsvollsten M...,Nothing_found,[Ihre Tätigkeit: In einem der größten und ansp...
...,...,...,...,...,...,...,...
880,\nRelayr,Data Scientist (m/f),Nothing_found,München,\nPerform ad-hoc data analysis and present res...,vor 9 Tagen,[Relayr is the fastest growing and one of the ...
881,\n\nNew Yorker,BI Data Analyst / Business Analyst (m/w/d),Nothing_found,Berlin-Schöneberg,\nFundierte quantitative und analytische Fähig...,vor 30+ Tagen,[Selbständige Durchführung von (Ad-Hoc)-Analys...
882,\nFintu Data Science GmbH,Werkstudent Data Visualization (d/w/m),Nothing_found,Berlin,\nDu konzipierst und entwickelst Visualisierun...,vor 30 Tagen,[Dein Job\nDu konzipierst und entwickelst Visu...
883,\nUNICEPTA GmbH,Data Analyst (m/w/d) Analytics & Insights in K...,Nothing_found,Nothing_found,\nDu arbeitest eng mit den Consultants und Dat...,Nothing_found,[Deine Aufgaben:\nMit Deiner Arbeit bist Du ma...


In [7]:
#establish how many ads had no salary
ads[ads['salary']=="Nothing_found"].shape

(157, 7)

In [249]:
#print data to csv
#ads.to_csv(r"data\indeed_scrape_data.csv")

In [14]:
i

461

In [15]:
today = datetime.now().strftime('%Y_%m_%d_%H_%M')
ads.to_csv(parent_folder+'/data/indeed_de_'+today+'.csv', index=True, sep='\t')